# Устанавливаем библиотеки

In [ ]:
!pip install datasets -qq

In [ ]:
!pip install transformers -qq

In [ ]:
!pip install accelerate -qq

In [ ]:
!pip install python-Levenshtein -qq

In [ ]:
!pip install fuzzywuzzy -qq

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from typing import Dict, List, Union

In [ ]:
from datasets import load_dataset
from fuzzywuzzy import fuzz, process
from sklearn.metrics import classification_report
from tqdm.notebook import tqdm
from transformers import pipeline

In [ ]:
MessageHint = Dict[str, Union[List[Dict[str, str]], List[List[Dict[str, str]]]]]

In [ ]:
def prepare_message_for_llm(text: Union[str, List[str]], categories: List[str]) -> MessageHint:
    """
    Функция добавляет к текстам prompt с инструкцией для LLM.

    """
    assert len(categories) >= 2, f'Expected 2 or more categories, got {len(categories)} ones.'

    categories_as_string = ', '.join(categories[:-1]) + ' и ' + categories[-1]
    if isinstance(text, str):
        prompt = f'Прочтите, пожалуйста, следующий текст и определите, какая тема из известного ' \
                 f'списка тем наиболее представлена в следующем тексте. ' \
                 f'В качестве ответа напишите только название темы из списка, больше ничего.\n' \
                 f'Список тем: {categories_as_string}.\nТекст: {" ".join(text.split())}\nВаш ответ: '
        messages = [
            {
                'role': 'system',
                'content':
                'Вы - полезный помощник, умеющий читать тексты на русском языке, глубоко понимать их и анализировать.'
            },
            {
                'role': 'user',
                'content': prompt
            }
        ]
    else:
        messages = []
        for it in text:
            prompt = f'Прочтите, пожалуйста, следующий текст и определите, какая тема из известного ' \
                     f'списка тем наиболее представлена в следующем тексте. ' \
                     f'В качестве ответа напишите только название темы из списка, больше ничего.\n' \
                     f'Список тем: {categories_as_string}.\nТекст: {" ".join(text.split())}\nВаш ответ: '
            messages.append([
                {
                    'role': 'system',
                    'content':
                    'Вы - полезный помощник, умеющий читать тексты на русском языке, глубоко понимать их и анализировать.'
                },
                {
                    'role': 'user',
                    'content': prompt
                }
            ])
    return {'message_for_llm': messages}

In [ ]:
test_message = """Крупнейшим дуалистом был великий французский философ, физик и математик Рене Декарт (1596–1650).
Дуализм не оформился в самостоятельное философское направление (как материализм и идеализм), поскольку столкнулся с
неразрешимой проблемой. Невозможно объяснить, как взаимодействуют материя и сознание, если они являются совершенно
различными самостоятельными субстанциями и не могут, следовательно, иметь ничего общего, никаких «точек касания».
В конечном итоге дуалисты вынуждены соглашаться либо с материалистическим, либо с идеалистическим пониманием мира.
""".replace('\n', '')
test_categories = ['философия', 'математика', 'физика']

prepare_message_for_llm(test_message, test_categories)

{'message_for_llm': [{'role': 'system',
   'content': 'Вы - полезный помощник, умеющий читать тексты на русском языке, глубоко понимать их и анализировать.'},
  {'role': 'user',
   'content': 'Прочтите, пожалуйста, следующий текст и определите, какая тема из известного списка тем наиболее представлена в следующем тексте. В качестве ответа напишите только название темы из списка, больше ничего.\nСписок тем: философия, математика и физика.\nТекст: Крупнейшим дуалистом был великий французский философ, физик и математик Рене Декарт (1596–1650).Дуализм не оформился в самостоятельное философское направление (как материализм и идеализм), поскольку столкнулся снеразрешимой проблемой. Невозможно объяснить, как взаимодействуют материя и сознание, если они являются совершенноразличными самостоятельными субстанциями и не могут, следовательно, иметь ничего общего, никаких «точек касания».В конечном итоге дуалисты вынуждены соглашаться либо с материалистическим, либо с идеалистическим пониманием мир

# Qwen2
Technical report - https://arxiv.org/pdf/2407.10671  
https://qwenlm.github.io/blog/qwen2/


* Размеры от 0.5B до 72B.
* 29 языков, включая русский.
* Поддержка контекстного окна до 128K токенов у Qwen2-7B-Instruct и Qwen2-72B-Instruct.
* Использование Grouped Query Attention (GQA).



In [ ]:
llm_pipeline = pipeline(model='Qwen/Qwen2-7B-Instruct', device_map='auto', torch_dtype='auto')

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/27.8k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

### Выгрузим данные
https://huggingface.co/datasets/Davlan/sib200/viewer/rus_Cyrl

In [ ]:
DATASET_NAME = 'Davlan/sib200'
DATASET_LANGUAGE = 'rus_Cyrl'
train_set = load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='train')
validation_set = load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='validation')
test_set = load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='test')

README.md:   0%|          | 0.00/47.9k [00:00<?, ?B/s]

data/rus_Cyrl/train.tsv:   0%|          | 0.00/195k [00:00<?, ?B/s]

data/rus_Cyrl/dev.tsv:   0%|          | 0.00/25.3k [00:00<?, ?B/s]

data/rus_Cyrl/test.tsv:   0%|          | 0.00/57.4k [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
list_of_categories = sorted(list(
    set(train_set['category']) | set(validation_set['category']) | set(test_set['category'])
))
print(f'Categories for classification are: {list_of_categories}')

Categories for classification are: ['entertainment', 'geography', 'health', 'politics', 'science/technology', 'sports', 'travel']


In [ ]:
print(validation_set)

Dataset({
    features: ['index_id', 'category', 'text'],
    num_rows: 99
})


In [ ]:
validation_set[:1]

{'index_id': [548],
 'category': ['sports'],
 'text': ['Если увеличить расстояние для бега с четверти до половины мили, скорость становится не так важна, тогда как выносливость превращается в абсолютную необходимость.']}

### Добавим prompt

In [ ]:
validation_set_for_llm = validation_set.map(lambda it: prepare_message_for_llm(it['text'], list_of_categories))
test_set_for_llm = test_set.map(lambda it: prepare_message_for_llm(it['text'], list_of_categories))

Map:   0%|          | 0/99 [00:00<?, ? examples/s]

Map:   0%|          | 0/204 [00:00<?, ? examples/s]

In [ ]:
print(validation_set_for_llm)

Dataset({
    features: ['index_id', 'category', 'text', 'message_for_llm'],
    num_rows: 99
})


In [ ]:
print(validation_set_for_llm['message_for_llm'][0])
print(validation_set_for_llm['text'][0])

[{'content': 'Вы - полезный помощник, умеющий читать тексты на русском языке, глубоко понимать их и анализировать.', 'role': 'system'}, {'content': 'Прочтите, пожалуйста, следующий текст и определите, какая тема из известного списка тем наиболее представлена в следующем тексте. В качестве ответа напишите только название темы из списка, больше ничего.\nСписок тем: entertainment, geography, health, politics, science/technology, sports и travel.\nТекст: Если увеличить расстояние для бега с четверти до половины мили, скорость становится не так важна, тогда как выносливость превращается в абсолютную необходимость.\nВаш ответ: ', 'role': 'user'}]
Если увеличить расстояние для бега с четверти до половины мили, скорость становится не так важна, тогда как выносливость превращается в абсолютную необходимость.


### Сгенерируем ответы модели

In [ ]:
y_pred = list(map(
    lambda x: llm_pipeline(x, max_new_tokens=10)[0]['generated_text'],
    tqdm(validation_set_for_llm['message_for_llm'])
))
y_true = validation_set['category']

  0%|          | 0/99 [00:00<?, ?it/s]

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [ ]:
print(y_pred[0])

[{'content': 'Вы - полезный помощник, умеющий читать тексты на русском языке, глубоко понимать их и анализировать.', 'role': 'system'}, {'content': 'Прочтите, пожалуйста, следующий текст и определите, какая тема из известного списка тем наиболее представлена в следующем тексте. В качестве ответа напишите только название темы из списка, больше ничего.\nСписок тем: entertainment, geography, health, politics, science/technology, sports и travel.\nТекст: Если увеличить расстояние для бега с четверти до половины мили, скорость становится не так важна, тогда как выносливость превращается в абсолютную необходимость.\nВаш ответ: ', 'role': 'user'}, {'role': 'assistant', 'content': 'sports'}]


### Результаты на валидации

In [ ]:
print(classification_report(y_true=y_true, y_pred=[x[-1]['content'] for x in y_pred]))

                    precision    recall  f1-score   support

           culture       0.00      0.00      0.00         0
     entertainment       0.57      0.44      0.50         9
         geography       0.44      0.50      0.47         8
            health       0.83      0.45      0.59        11
          politics       0.81      0.93      0.87        14
science/technology       0.72      0.92      0.81        25
            sports       0.92      0.92      0.92        12
         transport       0.00      0.00      0.00         0
    transportation       0.00      0.00      0.00         0
            travel       0.77      0.50      0.61        20

          accuracy                           0.71        99
         macro avg       0.51      0.47      0.48        99
      weighted avg       0.74      0.71      0.71        99



In [ ]:
len(set([x[-1]['content'] for x in y_pred]))

10

In [ ]:
print(set([x[-1]['content'] for x in y_pred]))
print(list_of_categories)
print(set([x[-1]['content'] for x in y_pred]) - set(list_of_categories))
print(set(list_of_categories) - set([x[-1]['content'] for x in y_pred]))

{'travel', 'sports', 'health', 'science/technology', 'transport', 'entertainment', 'geography', 'politics', 'transportation', 'culture'}
['entertainment', 'geography', 'health', 'politics', 'science/technology', 'sports', 'travel']
{'culture', 'transportation', 'transport'}
set()


**Модель сегенрировала лишние ответы**

In [ ]:
llm_pipeline(prepare_message_for_llm(test_message, test_categories)['message_for_llm'], max_new_tokens=500)

[{'generated_text': [{'role': 'system',
    'content': 'Вы - полезный помощник, умеющий читать тексты на русском языке, глубоко понимать их и анализировать.'},
   {'role': 'user',
    'content': 'Прочтите, пожалуйста, следующий текст и определите, какая тема из известного списка тем наиболее представлена в следующем тексте. В качестве ответа напишите только название темы из списка, больше ничего.\nСписок тем: философия, математика и физика.\nТекст: Крупнейшим дуалистом был великий французский философ, физик и математик Рене Декарт (1596–1650).Дуализм не оформился в самостоятельное философское направление (как материализм и идеализм), поскольку столкнулся снеразрешимой проблемой. Невозможно объяснить, как взаимодействуют материя и сознание, если они являются совершенноразличными самостоятельными субстанциями и не могут, следовательно, иметь ничего общего, никаких «точек касания».В конечном итоге дуалисты вынуждены соглашаться либо с материалистическим, либо с идеалистическим пониманием 

### Применим постобработку ответов модели
Воспользуемся расстоянием Левенштейна

In [ ]:
y_pred_with_normalization = list(map(
    lambda it: process.extractOne(it[-1]['content'], list_of_categories, scorer=fuzz.token_sort_ratio)[0],
    tqdm(y_pred)
))

  0%|          | 0/99 [00:00<?, ?it/s]

In [ ]:
print(y_pred_with_normalization[0])

sports


In [ ]:
print(classification_report(y_true=y_true, y_pred=y_pred_with_normalization))

                    precision    recall  f1-score   support

     entertainment       0.57      0.44      0.50         9
         geography       0.44      0.50      0.47         8
            health       0.83      0.45      0.59        11
          politics       0.81      0.93      0.87        14
science/technology       0.72      0.92      0.81        25
            sports       0.79      0.92      0.85        12
            travel       0.67      0.50      0.57        20

          accuracy                           0.71        99
         macro avg       0.69      0.67      0.66        99
      weighted avg       0.71      0.71      0.69        99



### Посмотрим результаты на тестовом подмножестве

In [ ]:
y_pred = list(map(
    lambda x: llm_pipeline(x, max_new_tokens=10)[0]['generated_text'],
    tqdm(test_set_for_llm['message_for_llm'])
))
y_true = test_set['category']

  0%|          | 0/204 [00:00<?, ?it/s]

In [ ]:
print(classification_report(y_true=y_true, y_pred=[x[-1]['content'] for x in y_pred]))

                    precision    recall  f1-score   support

         astronomy       0.00      0.00      0.00         0
     entertainment       0.83      0.53      0.65        19
         geography       0.80      0.71      0.75        17
            health       0.90      0.82      0.86        22
           history       0.00      0.00      0.00         0
        literature       0.00      0.00      0.00         0
          politics       0.66      0.97      0.78        30
          religion       0.00      0.00      0.00         0
science/technology       0.83      0.94      0.88        51
            sports       0.91      0.80      0.85        25
    transportation       0.00      0.00      0.00         0
            travel       0.96      0.65      0.78        40
           weather       0.00      0.00      0.00         0

          accuracy                           0.80       204
         macro avg       0.45      0.42      0.43       204
      weighted avg       0.85      0.8

### Также применим постобработку

In [ ]:
y_pred_with_normalization = list(map(
    lambda it: process.extractOne(it[-1]['content'], list_of_categories, scorer=fuzz.token_sort_ratio)[0],
    y_pred
))

In [ ]:
print(classification_report(y_true=y_true, y_pred=y_pred_with_normalization))

                    precision    recall  f1-score   support

     entertainment       0.83      0.53      0.65        19
         geography       0.80      0.71      0.75        17
            health       0.86      0.82      0.84        22
          politics       0.66      0.97      0.78        30
science/technology       0.81      0.94      0.87        51
            sports       0.83      0.80      0.82        25
            travel       0.93      0.68      0.78        40

          accuracy                           0.80       204
         macro avg       0.82      0.78      0.78       204
      weighted avg       0.82      0.80      0.80       204



# Обобщение вышеизложенных действий в функцию

In [ ]:
def get_predictions(model, dataset, normalize=False, cut_ss=False):
    def process_text(text):
        # Удаляем окончательные буквы 's' и приводим к нижнему регистру
        while text.endswith('s'):
            text = text[:-1]
        return text.lower()

    # Получаем предсказания от модели
    pred = list(map(
        lambda x: model(x, max_new_tokens=10)[0]['generated_text'][-1]['content'],
        tqdm(dataset['message_for_llm'])
    ))
    true = dataset['category']

    # Нормализация предсказаний и истинных значений
    if normalize == True:
      print("Normalize by Levenstein distance")
      pred = list(map(
        lambda it: process.extractOne(it, list_of_categories, scorer=fuzz.token_sort_ratio)[0],
        tqdm(pred)
      ))
    elif cut_ss == True:
      print("Normalize by cutting ss")
      pred = list(map(process_text, pred))
      true = list(map(process_text, dataset['category']))

    return pred, true

### Еще раз получаем результаты на валидации

In [ ]:
val_pred, val_true = get_predictions(llm_pipeline, validation_set_for_llm, normalize=True)
print(classification_report(y_true=val_true, y_pred=[x for x in val_pred]))

  0%|          | 0/99 [00:00<?, ?it/s]

Normalize by Levenstein distance


  0%|          | 0/99 [00:00<?, ?it/s]

                    precision    recall  f1-score   support

     entertainment       0.67      0.44      0.53         9
         geography       0.56      0.62      0.59         8
            health       0.80      0.36      0.50        11
          politics       0.76      0.93      0.84        14
science/technology       0.71      0.96      0.81        25
            sports       0.79      0.92      0.85        12
            travel       0.64      0.45      0.53        20

          accuracy                           0.71        99
         macro avg       0.70      0.67      0.66        99
      weighted avg       0.71      0.71      0.69        99



### Еще раз получаем результаты на тесте

In [ ]:
test_pred, test_true = get_predictions(llm_pipeline, test_set_for_llm, normalize=True)
print(classification_report(y_true=test_true, y_pred=[x for x in test_pred]))

  0%|          | 0/204 [00:00<?, ?it/s]

Normalize by Levenstein distance


  0%|          | 0/204 [00:00<?, ?it/s]

                    precision    recall  f1-score   support

     entertainment       0.83      0.53      0.65        19
         geography       0.80      0.71      0.75        17
            health       0.86      0.82      0.84        22
          politics       0.66      0.97      0.78        30
science/technology       0.80      0.94      0.86        51
            sports       0.83      0.80      0.82        25
            travel       0.96      0.68      0.79        40

          accuracy                           0.80       204
         macro avg       0.82      0.78      0.78       204
      weighted avg       0.82      0.80      0.80       204



# Изменяем промт и смотрим с ним результаты на валидации и тесте

In [ ]:
def new_prepare_message_for_llm(text: Union[str, List[str]], categories: List[str]) -> dict:
    """
    Функция добавляет к текстам prompt с инструкцией для LLM.

    """
    assert len(categories) >= 2, f'Expected 2 or more categories, got {len(categories)} ones.'
    categories_as_string = ', '.join(categories[:-1]) + ' и ' + categories[-1]
    base_prompt = (
        f'Прочтите следующий текст и определите, какая из перечисленных тем наиболее соответствует его содержанию. '
        f'Выберите только одну наиболее подходящую тему из списка и напишите только её название, ничего больше.\n'
        f'Список тем: {categories_as_string}.\nТекст: '
    )

    system_message = {
        'role': 'system',
        'content': 'Вы - полезный помощник, который глубоко анализирует тексты на русском языке и точно определяет основную тему.'
    }

    def create_message(text_item):
        prompt = f"{base_prompt}{' '.join(text_item.split())}\nВаш ответ: "
        return [
            system_message,
            {'role': 'user', 'content': prompt}
        ]

    if isinstance(text, str):
        messages = create_message(text)
    else:
        messages = [create_message(it) for it in text]

    return {'message_for_llm': messages}

In [ ]:
new_validation_set_for_llm = validation_set.map(lambda it: new_prepare_message_for_llm(it['text'], list_of_categories))
print(new_validation_set_for_llm)
print(new_validation_set_for_llm['message_for_llm'][0])
new_test_set_for_llm = test_set.map(lambda it: new_prepare_message_for_llm(it['text'], list_of_categories))
print(new_test_set_for_llm)
print(new_test_set_for_llm['message_for_llm'][0])

Map:   0%|          | 0/99 [00:00<?, ? examples/s]

Dataset({
    features: ['index_id', 'category', 'text', 'message_for_llm'],
    num_rows: 99
})
[{'content': 'Вы - полезный помощник, который глубоко анализирует тексты на русском языке и точно определяет основную тему.', 'role': 'system'}, {'content': 'Прочтите следующий текст и определите, какая из перечисленных тем наиболее соответствует его содержанию. Выберите только одну наиболее подходящую тему из списка и напишите только её название, ничего больше.\nСписок тем: entertainment, geography, health, politics, science/technology, sports и travel.\nТекст: Если увеличить расстояние для бега с четверти до половины мили, скорость становится не так важна, тогда как выносливость превращается в абсолютную необходимость.\nВаш ответ: ', 'role': 'user'}]


Map:   0%|          | 0/204 [00:00<?, ? examples/s]

Dataset({
    features: ['index_id', 'category', 'text', 'message_for_llm'],
    num_rows: 204
})
[{'content': 'Вы - полезный помощник, который глубоко анализирует тексты на русском языке и точно определяет основную тему.', 'role': 'system'}, {'content': 'Прочтите следующий текст и определите, какая из перечисленных тем наиболее соответствует его содержанию. Выберите только одну наиболее подходящую тему из списка и напишите только её название, ничего больше.\nСписок тем: entertainment, geography, health, politics, science/technology, sports и travel.\nТекст: Мутация вносит новую генетическую вариацию, в то время как отбор убирает её из набора проявляющихся вариаций.\nВаш ответ: ', 'role': 'user'}]


In [ ]:
val_pred, val_true = get_predictions(llm_pipeline, new_validation_set_for_llm, cut_ss=True)
print(classification_report(y_true=val_true, y_pred=[x for x in val_pred]))

  0%|          | 0/99 [00:00<?, ?it/s]

Normalize by cutting ss
                    precision    recall  f1-score   support

           culture       0.00      0.00      0.00         0
     entertainment       0.57      0.44      0.50         9
         geography       0.67      0.75      0.71         8
            health       0.86      0.55      0.67        11
           politic       0.78      1.00      0.88        14
science/technology       0.79      0.92      0.85        25
             sport       0.92      0.92      0.92        12
         transport       0.00      0.00      0.00         0
            travel       0.77      0.50      0.61        20

          accuracy                           0.75        99
         macro avg       0.59      0.56      0.57        99
      weighted avg       0.78      0.75      0.75        99



In [ ]:
test_pred, test_true = get_predictions(llm_pipeline, new_test_set_for_llm, cut_ss=True)
print(classification_report(y_true=test_true, y_pred=[x for x in test_pred]))

  0%|          | 0/204 [00:00<?, ?it/s]

Normalize by cutting ss
                    precision    recall  f1-score   support

         astronomy       0.00      0.00      0.00         0
         education       0.00      0.00      0.00         0
     entertainment       0.90      0.47      0.62        19
         geography       0.80      0.71      0.75        17
            health       0.85      0.77      0.81        22
        literature       0.00      0.00      0.00         0
           politic       0.63      0.97      0.76        30
science/technology       0.81      0.90      0.85        51
             sport       0.91      0.80      0.85        25
            travel       0.96      0.68      0.79        40
  visa_requirement       0.00      0.00      0.00         0
           weather       0.00      0.00      0.00         0

          accuracy                           0.78       204
         macro avg       0.49      0.44      0.45       204
      weighted avg       0.84      0.78      0.79       204

